### This script will generate climate statistics for individual SPECIES OBSERVATIONS from GLDAS for the EXPERIMENTAL SITES.
* Based on notes climate variables we decided on from here:
    https://github.com/AileneKane/radcliffe/wiki/Team-background:-Notes-from-6-April-2016
 
* Including:
        number of chilling days (1 Sept - 31 Dec of previous year)
        GDD (0/5) from 1 Jan to start date
        total amount of precipitation: January 1 to start date
 
* part of github project https://github.com/AileneKane/radcliffe

In [1]:
qtconsole

In [1]:
# Reset the environment (start clean)
%reset -f

# Import Modules and define functions
import calendar
import datetime
import os
import numpy as np
import numpy.ma as ma
import netCDF4
import matplotlib
import copy
from matplotlib import pyplot as plt
import scipy
import scipy.signal
import scipy.io as sio
import seaborn as sns
import pandas as pd
import math
import scipy.stats as stats
from IPython.display import display
from mpl_toolkits.basemap import Basemap, cm, maskoceans
import datetime as dt  # Python standard library datetime  module

# Embeds plots inside the notebook (use in iPython Notebook)
%matplotlib inline

# Month Vector
mons = np.arange(1,12+1)

# GLDAS Data Directory
dir_gldas  = '../Analyses/teambackground/output/SiteMet/'

# Output Directory
dir_output = '../Analyses/teambackground/output/obsclim/'

# List of GLDAS sites to calculate data from
#   OBSERVATIONS ONLY
site_names = [ \
    'concord', \
    'fargo', \
    'gothic', \
    'harvard', \
    'hubbard', \
    'konza', \
    'mikesell', \
    'mohonk', \
    'niwot', \
    'uwm', \
    'washdc']


### Load experimental phenological data.

In [2]:
# Load the csv file into a dataframe
df_exppheno = pd.read_csv('../Analyses/obsphenoBIC.csv')

# Pull out site codes, doy, year, phenophase event, genus, species
exp_site    = df_exppheno.site
exp_plot    = df_exppheno.plt
exp_event   = df_exppheno.event
exp_year    = df_exppheno.year
exp_genus   = df_exppheno.genus
exp_species = df_exppheno.species
exp_doy     = df_exppheno.doy

# Print out unique site names
np.unique(exp_site)

array(['bolmgren', 'concord', 'fargo', 'fitter', 'gothic', 'harvard',
       'hubbard', 'konza', 'marsham', 'mikesell', 'mohonk', 'niwot',
       'rousi', 'uwm', 'washdc'], dtype=object)

### Loop through each site, and then calculate statistics for each individual record at that site

In [3]:
# FROM GLDAS:
# For the current site and each year, calculate daily GDD0, GDD5, and OND Chill Days
for i_rec in enumerate(site_names):
      
    # Print current site
    curr_site = site_names[i_rec[0]]
    print(curr_site)
    
    # Find Location for current site
    i_site = np.where(exp_site==curr_site)[0]
    
    # Pull out current site info
    yrs_site     = np.float64(exp_year[i_site])
    doy_site     = np.float64(exp_doy[i_site])
   
    # Open Climate Data for this site
    df_currsite = pd.read_csv(dir_gldas+curr_site+'_gldas_2.0_1949-2010.csv')
    
    # Pull out and create a vector of unique years
    yrs_all  = np.array(df_currsite.year)
    yrs_uniq = np.unique(yrs_all)
    
    # FOR EACH YEAR OF GLDAS, CALCULATE DAILY GDD AND SEASONAL CHILL
    gdd0_site     = np.zeros((yrs_uniq.size,365))
    gdd5_site     = np.zeros((yrs_uniq.size,365))
    chillOND_site = np.zeros((yrs_uniq.size))
    
    for i_yr in enumerate(yrs_uniq):
        
        # Current year
        curr_yr = i_yr[1]
        
        # Location of Current Year
        loc_yr = df_currsite.year==curr_yr
    
        # Month vector 
        month_vect = df_currsite.mo[loc_yr]
    
        # Pull out values for current year
        tmean_curr = np.array(df_currsite.tmean[loc_yr])-273.15
        
        # Calculate GDD, base ZERO
        gdd0                          = copy.deepcopy(tmean_curr)
        gdd0[np.where(tmean_curr<=0)] = 0
                
        # Calculate GDD, base FIVE
        gdd5                          = copy.deepcopy(tmean_curr)
        gdd5[np.where(tmean_curr<5)]  = 0
        gdd5[np.where(tmean_curr>=5)] = gdd5[np.where(tmean_curr>=5)]-5

        # Calculate OND Chill Days
        i_OND     = np.where(month_vect>=10)
        tmean_OND = tmean_curr[i_OND]
        chill_OND = np.where(tmean_OND<5)[0].size

        # Store for analyses
        gdd0_site[i_yr[0],0:365]     = gdd0[0:365]
        gdd5_site[i_yr[0],0:365]     = gdd5[0:365]
        chillOND_site[i_yr[0]]       = chill_OND

    # NOW THAT GLDAS STATS HAVE BEEN CALCULATED
    # Loop through each phenological observation, and calculate cumulative GDDs and chill days for that year+day

    # Initialize Storage Arrays
    pheno_site_gdd0     = np.zeros(yrs_site.size)
    pheno_site_gdd5     = np.zeros(yrs_site.size)
    pheno_site_chillOND = np.zeros(yrs_site.size)

    # Loop through each phenological observation
    for i_pheno in enumerate(yrs_site):
        
        # Find current year and date for phenological observation
        pheno_yr  = np.int(yrs_site[i_pheno[0]])
        pheno_doy = np.int(doy_site[i_pheno[0]])
        
        # Assign NaN value if no climate data available
        # Other, make the GDD and Chill Day Calculation
        if pheno_yr>2010:
            pheno_site_gdd0[i_pheno[0]]     = np.nan
            pheno_site_gdd5[i_pheno[0]]     = np.nan
            pheno_site_chillOND[i_pheno[0]] = np.nan
        elif pheno_yr<1949:
            pheno_site_gdd0[i_pheno[0]]     = np.nan
            pheno_site_gdd5[i_pheno[0]]     = np.nan
            pheno_site_chillOND[i_pheno[0]] = np.nan
        else:
            # Index Locations
            i_gdd_yr  = np.where(yrs_uniq==[pheno_yr])[0]
            i_gdd_doy = np.arange(0,pheno_doy)
        
            # Store Data
            pheno_site_gdd0[i_pheno[0]]     = np.sum(gdd0_site[i_gdd_yr,i_gdd_doy])
            pheno_site_gdd5[i_pheno[0]]     = np.sum(gdd5_site[i_gdd_yr,i_gdd_doy])
            pheno_site_chillOND[i_pheno[0]] = chillOND_site[i_gdd_yr-1]
    
    # CREATE DATAFRAME AND EXPORT
    site_site    = exp_site[i_site]
    genus_site   = exp_genus[i_site]
    species_site = exp_species[i_site]
    event_site   = exp_event[i_site]
    plot_site    = exp_plot[i_site]

    # Populate Dataframe
    header_txt    = ['site','genus','species','event','year','doy','GDD0','GDD5','ChillDaysOND','plt']
    df_out              = pd.DataFrame(columns=header_txt)
    df_out.site         = site_site
    df_out.genus        = genus_site
    df_out.species      = species_site
    df_out.event        = event_site
    df_out.year         = yrs_site
    df_out.doy          = doy_site
    df_out.GDD0         = pheno_site_gdd0
    df_out.GDD5         = pheno_site_gdd5
    df_out.ChillDaysOND = pheno_site_chillOND
    df_out.plt          = plot_site

    # Save to File
    outfile       = dir_output+curr_site+'_pheno_clim.csv' # name of output file
    df_out.to_csv(outfile,sep=',')            # save to file 


concord
fargo
gothic
harvard
hubbard
konza
mikesell
mohonk
niwot
uwm
washdc
